In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO


In [3]:
import requests
from urllib import parse

def get_data(keyword):
    enc = parse.quote(keyword)
    url = f"http://172.31.151.225:5000/api/search/uat?keyword={enc}&page=1&size=200&age=0&gender=&accuracy="
    return requests.get(url=url).json()['products']

In [30]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT a.keyword,
       a.bid_price,
       b.id as ad_set_product_id,
       b.ad_set_id,
       b.creator_uid AS UID
FROM service1_quicket.ad_set_product_bid_keyword a
JOIN service1_quicket.ad_set_product b ON a.ad_set_product_id = b.id
WHERE b.status = 3
'''

keywords = pd.read_sql(q, con=bun_dw) 

keywords['ad_set_product_id'] = keywords['ad_set_product_id'].astype(str)

ad_set_product_ids = keywords.ad_set_product_id.unique().tolist()
total_num = len(ad_set_product_ids)
print('total length: {}'.format(total_num))
df_pre = {}
for num, i in enumerate(ad_set_product_ids):
    df_pre[i] = {}
    if num % 50 == 0:
        print(num)
    if num == 500:
        break
    bid_keywords = keywords[keywords['ad_set_product_id'] == str(i)]['keyword'].unique()
    for keyword in bid_keywords:
        results = get_data(keyword)
        for rank, ad in enumerate(results):
            if str(ad['adSetProductId']) == str(i):
                df_pre[i][keyword]=rank
                break

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 10:26:51
total length: 3978
0
50
100
150
200
250
300
350
400
450
500


In [31]:
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 10:41:05


In [33]:
import json
with open('pre_data_2103261041.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(df_pre, ensure_ascii=False))

In [46]:
df_pre2 = {}
for product_id in df_pre:
    if len(df_pre[product_id]) > 0:
        df_pre2[product_id] = df_pre[product_id]

In [37]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT a.keyword,
       a.bid_price,
       b.id as ad_set_product_id,
       b.ad_set_id,
       b.creator_uid AS UID
FROM service1_quicket.ad_set_product_bid_keyword a
JOIN service1_quicket.ad_set_product b ON a.ad_set_product_id = b.id
WHERE b.status = 3
'''

keywords = pd.read_sql(q, con=bun_dw) 

keywords['ad_set_product_id'] = keywords['ad_set_product_id'].astype(str)

ad_set_product_ids = keywords.ad_set_product_id.unique().tolist()
total_num = len(ad_set_product_ids)
print('total length: {}'.format(total_num))
df_20m = {}
for num, i in enumerate(ad_set_product_ids):
    df_20m[i] = {}
    if num % 50 == 0:
        print(num)
    if num == 500:
        break
    bid_keywords = keywords[keywords['ad_set_product_id'] == str(i)]['keyword'].unique()
    for keyword in bid_keywords:
        results = get_data(keyword)
        for rank, ad in enumerate(results):
            if str(ad['adSetProductId']) == str(i):
                df_20m[i][keyword]=rank
                break

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 11:02:47
total length: 3959
0
50
100
150
200
250
300
350


KeyboardInterrupt: 

In [42]:
with open('20m_data_21032611.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(df_20m, ensure_ascii=False))
        
df_20m2 = {}
for product_id in df_20m:
    if len(df_20m[product_id]) > 0:
        df_20m2[product_id] = df_20m[product_id]

In [47]:
both = []
for i in df_pre2:
    for j in df_20m2:
        if i == j:
            both.append(i)
both = list(set(both))

In [48]:
len(both)

175

In [50]:
compare = {}
for product_id in both:
    compare[product_id] = {}
    for keyword in df_pre2[product_id]:
        pre_rank = df_pre2[product_id][keyword]
        if keyword in df_20m2[product_id]:
            post_rank = df_20m2[product_id][keyword]
            compare[product_id][keyword] = [pre_rank, post_rank, pre_rank-post_rank]
        else:
            post_rank = None
            compare[product_id][keyword] = [pre_rank, post_rank, None]

In [52]:
with open('compare_20m.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(compare, ensure_ascii=False))

In [89]:
def compare(json, diff):
    compare_dict = {}
    for key, bids in json.items():
        size = len(bids)
        tmp = compare_dict.get(size, [])
        cnt = 0
        for bid_keyword, values in bids.items():
            if values[2]:
                if values[2] <= diff:
                    cnt += 1
        tmp.append(cnt)
        compare_dict[size] = tmp
    return compare_dict


In [79]:
f = open('compare_20m.json', 'r', encoding='UTF-8-sig')

compare_30m = json.load(f)

In [116]:
f = open('compare_AC.json', 'r', encoding='UTF-8-sig')

compare_ac = json.load(f)

In [135]:
f = open('compare_AD.json', 'r', encoding='UTF-8-sig')

compare_ad = json.load(f)

In [149]:
len(compare_ad)

240

In [136]:
d = pd.DataFrame.from_dict(compare(compare_ad, -6), orient='index')

In [140]:
d.shape

(24, 64)

In [148]:
dt = d.T
dt[sorted(dt.columns)].to_csv('d.csv')

In [151]:
dt[5].groupby()

0     1.0
1     1.0
2     2.0
3     0.0
4     1.0
     ... 
59    1.0
60    0.0
61    0.0
62    2.0
63    1.0
Name: 5, Length: 64, dtype: float64

In [129]:
b.T.describe()

,6,5,4,17,8,9,18,7,20,1,...,3,21,19,11,10,24,12,14,16,23
count,38.000000,66.000000,19.000000,2.0,7.000000,6.0,3.0,15.000000,3.000000,11.0,...,10.000000,1.0,5.000000,3.0,5.0,1.0,1.0,2.0,1.0,1.0
mean,0.157895,0.424242,0.157895,0.0,0.285714,0.0,0.0,0.133333,2.333333,0.0,...,0.100000,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,1.0
std,0.436591,0.745460,0.374634,0.0,0.487950,0.0,0.0,0.351866,4.041452,0.0,...,0.316228,NaN,0.447214,0.0,0.0,NaN,NaN,0.0,NaN,NaN
min,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0
25%,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0
50%,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0
75%,0.000000,1.000000,0.000000,0.0,0.500000,0.0,0.0,0.000000,3.500000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0
max,2.000000,2.000000,1.000000,0.0,1.000000,0.0,0.0,1.000000,7.000000,0.0,...,1.000000,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [94]:
a = pd.DataFrame.from_dict(compare(compare_30m, -6), orient='index').T
a

,2,6,17,4,8,9,18,7,20,5,19,3,24,10,1,14,16,23,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN
2,0.0,0.0,NaN,0.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN
3,0.0,0.0,NaN,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
4,0.0,0.0,NaN,0.0,0.0,NaN,NaN,1.0,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
for col in sorted(d.T.columns):
    t = d.T[col]
    tmp_cnt = 0
    for i in t:
        if i > 0:
            tmp_cnt += 1
    print(col, tmp_cnt)

1 0
2 0
3 1
4 1
5 30
6 15
7 3
8 3
9 3
10 0
11 0
12 0
14 0
16 0
17 0
18 0
19 1
20 2
21 1
22 7
23 1
24 1
25 0
26 1


In [115]:
a.to_csv('a.csv')

In [100]:
aa = a.notna().sum().reset_index()
aa.columns = ['bid_keyword_cnt', 'over_cnt']
aa.sort_values('bid_keyword_cnt')

,bid_keyword_cnt,over_cnt
14,1,5
0,2,13
11,3,3
3,4,10
9,5,62
1,6,38
7,7,13
4,8,7
5,9,4
13,10,2


In [54]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT a.keyword,
       a.bid_price,
       b.id as ad_set_product_id,
       b.ad_set_id,
       b.creator_uid AS UID
FROM service1_quicket.ad_set_product_bid_keyword a
JOIN service1_quicket.ad_set_product b ON a.ad_set_product_id = b.id
WHERE b.status = 3
'''

keywords = pd.read_sql(q, con=bun_dw) 

keywords['ad_set_product_id'] = keywords['ad_set_product_id'].astype(str)

ad_set_product_ids = keywords.ad_set_product_id.unique().tolist()
total_num = len(ad_set_product_ids)
print('total length: {}'.format(total_num))
df_2hr = {}
for num, i in enumerate(ad_set_product_ids):
    df_2hr[i] = {}
    if num % 50 == 0:
        print(num)
    if num == 500:
        break
    bid_keywords = keywords[keywords['ad_set_product_id'] == str(i)]['keyword'].unique()
    for keyword in bid_keywords:
        results = get_data(keyword)
        for rank, ad in enumerate(results):
            if str(ad['adSetProductId']) == str(i):
                df_2hr[i][keyword]=rank
                break

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 12:29:31
total length: 3947
0
50
100
150
200
250
300
350
400
450
500
Current Time = 12:39:56


In [56]:
with open('2hr_data_21032612.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(df_2hr, ensure_ascii=False))
        
df_2hr2 = {}
for product_id in df_2hr:
    if len(df_2hr[product_id]) > 0:
        df_2hr2[product_id] = df_2hr[product_id]

In [60]:
both = []
for i in df_pre2:
    for j in df_2hr2:
        if i == j:
            both.append(i)
both = list(set(both))

compare = {}
for product_id in both:
    compare[product_id] = {}
    for keyword in df_pre2[product_id]:
        pre_rank = df_pre2[product_id][keyword]
        if keyword in df_2hr2[product_id]:
            post_rank = df_2hr2[product_id][keyword]
            compare[product_id][keyword] = [pre_rank, post_rank, pre_rank-post_rank]
        else:
            post_rank = None
            compare[product_id][keyword] = [pre_rank, post_rank, None]

with open('compare_2hr.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(compare, ensure_ascii=False))

In [69]:
compare_ = {}
for key, bids in compare.items():
    size = len(bids)
    tmp = compare_.get(size, [])
    cnt = 0
    for bid_keyword, values in bids.items():
        if values[2] <= -6:
            cnt += 1
    if cnt > 0:
        tmp.append(cnt)
    compare_[size] = tmp

In [70]:
compare_

{1: [],
 2: [],
 3: [1],
 4: [1, 1, 1],
 5: [1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1],
 6: [1, 1, 1, 1, 2],
 7: [1, 1],
 8: [1, 1],
 9: [],
 10: [],
 11: [],
 12: [],
 14: [],
 16: [],
 17: [],
 18: [],
 19: [1],
 20: [7],
 21: [],
 23: [1],
 24: []}

In [71]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

q = '''
SELECT a.keyword,
       a.bid_price,
       b.id as ad_set_product_id,
       b.ad_set_id,
       b.creator_uid AS UID
FROM service1_quicket.ad_set_product_bid_keyword a
JOIN service1_quicket.ad_set_product b ON a.ad_set_product_id = b.id
WHERE b.status = 3
'''

keywords = pd.read_sql(q, con=bun_dw) 

keywords['ad_set_product_id'] = keywords['ad_set_product_id'].astype(str)

ad_set_product_ids = keywords.ad_set_product_id.unique().tolist()
total_num = len(ad_set_product_ids)
print('total length: {}'.format(total_num))
df_3hr30min = {}
for num, i in enumerate(ad_set_product_ids):
    df_3hr30min[i] = {}
    if num % 50 == 0:
        print(num)
    if num == 500:
        break
    bid_keywords = keywords[keywords['ad_set_product_id'] == str(i)]['keyword'].unique()
    for keyword in bid_keywords:
        results = get_data(keyword)
        for rank, ad in enumerate(results):
            if str(ad['adSetProductId']) == str(i):
                df_3hr30min[i][keyword]=rank
                break

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 14:11:55
total length: 3869
0
50
100
150
200
250
300
350
400
450
500
Current Time = 14:26:28


In [72]:
with open('3hr30min_data_21032612.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(df_3hr30min, ensure_ascii=False))
        
df_3hr30min2 = {}
for product_id in df_3hr30min:
    if len(df_3hr30min[product_id]) > 0:
        df_3hr30min2[product_id] = df_3hr30min[product_id]

In [74]:
both = []
for i in df_pre2:
    for j in df_3hr30min2:
        if i == j:
            both.append(i)
both = list(set(both))

compare = {}
for product_id in both:
    compare[product_id] = {}
    for keyword in df_pre2[product_id]:
        pre_rank = df_pre2[product_id][keyword]
        if keyword in df_3hr30min2[product_id]:
            post_rank = df_3hr30min2[product_id][keyword]
            compare[product_id][keyword] = [pre_rank, post_rank, pre_rank-post_rank]
        else:
            post_rank = None
            compare[product_id][keyword] = [pre_rank, post_rank, None]

with open('compare_3hr30min.json', 'w', encoding='UTF-8-sig') as file:
     file.write(json.dumps(compare, ensure_ascii=False))

In [76]:
compare__ = {}
for key, bids in compare.items():
    size = len(bids)
    tmp = compare__.get(size, [])
    cnt = 0
    for bid_keyword, values in bids.items():
        if values[2] <= -6:
            cnt += 1
    if cnt > 0:
        tmp.append(cnt)
    compare__[size] = tmp

In [77]:
compare__

{1: [],
 2: [],
 3: [1],
 4: [2],
 5: [1,
  1,
  2,
  1,
  1,
  2,
  2,
  2,
  2,
  3,
  2,
  1,
  1,
  2,
  1,
  1,
  1,
  2,
  1,
  2,
  2,
  2,
  2,
  2,
  1,
  2,
  2,
  1,
  2,
  1],
 6: [2, 3, 2, 1, 2, 2, 1, 1, 1, 1, 2, 4, 2, 2, 2],
 7: [1, 2, 2],
 8: [1, 1, 1],
 9: [1, 1, 2],
 10: [],
 11: [],
 12: [],
 14: [],
 16: [],
 17: [],
 18: [],
 19: [3],
 20: [7, 1],
 21: [17],
 22: [2, 2, 13, 10, 6, 6, 8],
 23: [16],
 24: [3],
 25: [],
 26: [9]}